# 1. Initializations

## 1.1 General imports

In [ ]:
### Data management
import pandas as pd
import numpy as np
import random

### Machine Learning

# transformation
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, OneHotEncoder

# models
from sklearn.model_selection import train_test_split

# metrics and evaluation
from scipy.stats import probplot

### Data Viz

# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline

# graphical seaborn
import seaborn as sns

# # graphical plotly
# import plotly.graph_objects as go
# import plotly.express as px
# # for jupyter notebook display management
# import plotly.io as pio
# pio.renderers.default = "notebook"


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General Classification functions

In [ ]:
import smartcheck.classification_common as cls

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.1.1 VELO COMPTAGE (Main Data Set)

#### Loading and column management (columns names normalization)

In [ ]:
df_cpt_velo_raw = dfc.load_dataset_from_config('velo_comptage_data', sep=';')

if df_cpt_velo_raw is not None and isinstance(df_cpt_velo_raw, pd.DataFrame):
    df_cpt_velo = dfc.normalize_column_names(df_cpt_velo_raw)

>**Rapport**
>
>On normalise les noms de colonnes du data set afin de faciliter leur manipulation dans le code. La norme utilisée est de type **snake_case**
>  - Remplacement des caractères spéciaux et espaces par des underscores
>  - Conversion en minuscules

In [ ]:
display(df_cpt_velo.head())
dfc.log_general_info(df_cpt_velo)
nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velo)
if nb_first != nb_total:
    print(dfc.duplicates_index_map(df_cpt_velo))

>**Rapport**
>
>Le dataset ***comptage velo*** présente globalement **942554** observations (lignes) sur **16** variables (colonnes)
>  - Il ne possède aucune observation en doublon ou sans aucune information.
>  - La proportion d'informations manquantes (détail par variable ci-dessous) est 3.38%
>
>| Variable | % informations manquantes |
>|-|-|
>| identifiant_du_compteur | 3.98% |
>| identifiant_du_site_de_comptage | 3.98% |
>| nom_du_site_de_comptage | 3.98% |
>| date_d_installation_du_site_de_comptage | 3.98% |
>| lien_vers_photo_du_site_de_comptage | 5.02% |
>| coordonnees_geographiques | 3.98% |
>| identifiant_technique_compteur | 5.02% |
>| id_photos | 5.02% |
>| test_lien_vers_photos_du_site_de_comptage | 5.02% |
>| id_photo_1 | 5.02% |
>| url_sites | 3.98% |
>| type_dimage | 5.02% |

#### Global description and correlation

In [ ]:
df_cpt_velo.info()
display(df_cpt_velo.head())
df_cpt_velo_desc = df_cpt_velo.select_dtypes(include=np.number).describe()
display(df_cpt_velo_desc)
df_cpt_velo_cr = df_cpt_velo.select_dtypes(include=np.number).corr()
display(df_cpt_velo_cr)

## 2.2 Data quality refinement

### 2.2.1 VELO COMPTAGE (Main Dataset)

In [ ]:
# Original backup and duplicates management
df_cpt_velib_orig = df_cpt_velo.copy()
df_cpt_velo = df_cpt_velo.drop_duplicates()

# 3. Data Viz' and Analysis

## 3.1 General Data Viz'

In [ ]:
# Vérificationn graphique de la répartition en loi normale de chaque données numérique
for col in df_cpt_velo.select_dtypes(include='number').columns:
    probplot(df_cpt_velo[col], dist="norm", plot=plt)
    plt.suptitle(f"Column {col}")
    plt.show()

## 3.2 Quantitative mono variable distribution

## 3.3 Qualitative mono variable distribution

## 3.4 Qualitative multi variable distribution

## 3.5 Quantitative multi variable correlation

# 4. Division in Train/Test

In [ ]:
# df_disp_velib = df_disp_velib_orig.copy()

In [ ]:
# (temporaire) Ajustement : enlever les variables non retravaillées pour le moment
df_disp_velib = df_cpt_velo.drop(columns=['identifiant_station',
                                            'nom_station', 
                                            'actualisation_de_la_donnee', 
                                            'coordonnees_geographiques', 
                                            'nom_communes_equipees',
                                            'code_insee_communes_equipees',
                                            'station_opening_hours'])

In [ ]:
# Separation features (X) et target (y) pour train et test
target_col = 'station_en_fonctionnement'
features = df_cpt_velo.drop(target_col, axis=1)
target = df_cpt_velo[target_col]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=66)
print("Train Set (X/y):", X_train.shape, y_train.shape)
print("Test Set (X/y):", X_test.shape, y_test.shape)

# 5. Feature engineering
Règle d'or : Toute opération qui "apprend" des données (i.e. utilise l’ensemble des valeurs pour calculer quelque chose) doit être faite après le split train/test — c’est-à-dire uniquement sur le train.

| Type de transformation                                                                                    | À faire avant le split ?                    | Détails                                                            |
| --------------------------------------------------------------------------------------------------------- | ------------------------------------------- | ------------------------------------------------------------------ |
| ✅ Création de features basées sur les colonnes existantes (ex: `BMI = weight / height²`)                  | **Avant**                                   | Pas de risque de fuite car c’est purement déterministe.            |
| ⚠️ Calculs dépendant de la distribution (moyennes, encodage fréquentiel, imputation par la médiane, etc.) | **Après** (sur le train uniquement)         | Risque de fuite de données si appliqué sur l’ensemble avant split. |
| ✅ Ajout de features exogènes fixes (données météo, géographiques, calendaires, etc.)                      | **Avant**                                   | Pas de dépendance au `target` ni à la répartition train/test.      |
| ⚠️ Encoding (`LabelEncoder`, `OneHot`, `TargetEncoding`, etc.)                                            | **Fit sur train, transform sur train/test** | Toujours fitter uniquement sur le `train`.                         |
| ⚠️ Standardisation / normalisation (Scaler)                                                               | **Fit sur train, transform sur train/test** | Pareil : `.fit()` sur train, `.transform()` sur test.              |


## 5.1 Modification localisées sur les variables d'entrainement

In [ ]:
# Exemple de modification localisée en fonction de la proximité à la médiane d'autre variables
# mask = (
#     (train['Gender'].isna()) &
#     (abs(train['Age'] - 30) > abs(train['Age'] - 41)) & # L’âge est plus proche de 41 que de 30
#     (train['Previously_Insured'] == 0) & # La personne n’était pas assurée auparavant
#     (train['Vehicle_Damage'] == 1) # Elle a subi un dommage sur son véhicule
# )
# train.loc[mask, 'Gender'] = 0

In [ ]:
# Exemple de modification par répartition spécifique entre deux valeurs 0 et 1
# proportion_tab = [0] * 55 + [1] * 45
# mask = (
#     (train['Gender'].isna()) &
# )
# train.loc[mask, 'Gender'] = train.loc[mask, 'Gender'].apply(lambda x: random.choice(proportion))

## 5.2 Preprocessing

### 5.2.1 Scaling (données quantitatives)

In [ ]:
# - ni outlier ni distribution loi normale : min/max
# - sans outlier mais distribution loi normale : standard
# - avec outlier : Robust 
mm_scal = MinMaxScaler()
r_scal = RobustScaler()
s_scal = StandardScaler()

r_scal_col = ['']
df_disp_velib[r_scal_col] = s_scal.fit_transform(df_disp_velib[r_scal_col])

### 5.2.1 Encoding (données qualitatives)

In [ ]:
# Technique                 Type                Colonnes créées     Principe
# get_dummies()	            Nominale	        N (ou N–1)	        Binaire par modalité
# OneHotEncoder	            Nominale, Cyclique	N	                Colonne 0/1 par modalité
# Sum Encoding	            Nominale	        N–1	                Différence avec moyenne globale
# Helmert Encoding	        Nominale	        N–1	                Contraste avec moyenne des modalités précédentes
# Backward Difference	    Ordinale	        N–1	                Contraste avec moyenne des modalités suivantes
# Binary Encoding	        Nominale	        log₂(N)	            Encodage binaire de l’index
# Hashing Encoding	        Nominale	        n_components	    Hash des modalités sur colonnes fixes
# Label Encoding	        Ordinale	        1	                Entier arbitraire
# Ordinal Encoding	        Ordinale	        1	                Rang croissant des modalités
# Target Encoding	        Nominale/Ordinale	1	                Moyenne de la cible par modalité
# Mean Encoding	            Nominale/Ordinale	1	                Idem Target Encoding
# Frequency Encoding	    Nominale/Ordinale	1	                Fréquence d'apparition
# Leave-One-Out	            Nominale/Ordinale	1	                Moyenne de la cible, sauf ligne courante
# James-Stein Encoding	    Nominale/Ordinale	1	                Moyenne pondérée par variance intercatégorie
# M-Estimate Encoding	    Nominale/Ordinale	1	                Moyenne cible lissée vers moyenne globale
# Probability Ratio	        Ordinale, binaire	1	                Log du ratio de probas classe 1 / classe 0
# WOE Encoding	            Ordinale, binaire	1	                Log( %positif / %négatif )
# Thermometer Encoding	    Ordinale	        N	                1 si la modalité est ≤ à une valeur
# Trigonométrique (sin/cos)	Cyclique	        2	                Encode la cyclicité
# Fourier / Radial	        Cyclique	        Variable	        Approximation périodique (base)
ohe_enc_col = ['']
ohe_enc = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
# Appliquer OneHotEncoder
X_train_enc_cat = ohe_enc.fit_transform(X_train[ohe_enc_col])
X_test_enc_cat = ohe_enc.transform(X_test[ohe_enc_col])
# Ajout des colonnes encodées à un DataFrame car le resultat de enc.fit/transform estun ndarray sans index/colonnes
X_train_cat_df = pd.DataFrame(X_train_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_train.index)
X_test_cat_df = pd.DataFrame(X_test_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_test.index)
# Suppression des colonnes catégoriques originales et ajout des colonnes encodées
X_train = pd.concat([X_train.drop(columns=ohe_enc_col), X_train_cat_df], axis=1)
X_test = pd.concat([X_test.drop(columns=ohe_enc_col), X_test_cat_df], axis=1)